<a href="https://colab.research.google.com/github/maxim920/Machine-Learning/blob/main/Kosenko_lab4_2_4_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer

drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/titanic.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
# Попередня обробка даних
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].value_counts().idxmax(), inplace=True)
df.drop('Cabin', axis=1, inplace=True)
df['Sex'] = df['Sex'].replace({'female': 1, 'male': 0})

# Перевірка, чи все добре після обробки
print("\nПропущені значення після обробки:")
print(df.isnull().sum())


Пропущені значення після обробки:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


In [62]:

# Відбір необхідних стовпців
X = df[['Pclass', 'Sex', 'Age', 'Fare']]
y = df['Survived']


In [63]:
# Розділення даних
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [64]:
# Створення моделей класифікації
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier()
}

In [65]:
# Параметри для GridSearchCV
params = {
    "Logistic Regression": {"C": [0.1, 1, 10], "solver": ['liblinear', 'lbfgs']},
    "Decision Tree": {"max_depth": [3, 5, 7], "min_samples_split": [2, 5, 10]},
    "Random Forest": {"n_estimators": [50, 100, 200], "max_depth": [3, 5, 7], "min_samples_split": [2, 5, 10]}
}

In [66]:
# Навчання моделей та пошук оптимальних параметрів
best_models = {}
for name, model in models.items():
    print(f"\nНавчання моделі: {name}...")
    grid_search = GridSearchCV(model, params[name], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Найкращі параметри для {name}: {grid_search.best_params_}")


Навчання моделі: Logistic Regression...
Найкращі параметри для Logistic Regression: {'C': 0.1, 'solver': 'liblinear'}

Навчання моделі: Decision Tree...
Найкращі параметри для Decision Tree: {'max_depth': 3, 'min_samples_split': 2}

Навчання моделі: Random Forest...
Найкращі параметри для Random Forest: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50}


In [67]:
# Оцінка моделей
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"\n{name} - Оцінка моделі:")
    print(f"Точність: {accuracy_score(y_test, y_pred)}")
    print(f"Матриця помилок:\n{confusion_matrix(y_test, y_pred)}")
    print(f"Звіт про класифікацію:\n{classification_report(y_test, y_pred)}")


Logistic Regression - Оцінка моделі:
Точність: 1.0
Матриця помилок:
[[50  0]
 [ 0 34]]
Звіт про класифікацію:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84


Decision Tree - Оцінка моделі:
Точність: 1.0
Матриця помилок:
[[50  0]
 [ 0 34]]
Звіт про класифікацію:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84


Random Forest - Оцінка моделі:
Точність: 1.0
Матриця помилок:
[[50  0]
 [ 0 34]]
Звіт про класифікацію:
              precision  

In [68]:
best_model = max(best_models, key=lambda name: accuracy_score(y_test, best_models[name].predict(X_test)))
print(f"\nНайкраща модель: {best_model}")



Найкраща модель: Logistic Regression


In [69]:
# Прогнозування для 10 випадків
best_model_instance = best_models[best_model]
predictions = best_model_instance.predict(X_test[:10])
print(f"Прогноз для 10 випадків:\n{predictions}")

Прогноз для 10 випадків:
[0 1 0 0 1 0 1 0 1 0]


**Висновки**
У ході роботи з датасетом Titanic були побудовані три моделі класифікації: логістична регресія, дерево рішень та випадковий ліс. Для кожної моделі було підібрано оптимальні параметри за допомогою GridSearchCV. Для Logistic Regression найкращими параметрами стали {'C': 0.1, 'solver': 'liblinear'}, для Decision Tree — {'max_depth': 3, 'min_samples_split': 2}, а для Random Forest — {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50}. Усі три моделі показали ідентичну точність, яка дорівнює 1.0. Матриця помилок для кожної моделі не показала жодних помилкових передбачень: усі 50 зразків класу 0 та 34 зразки класу 1 були передбачені правильно. Звіт про класифікацію для кожної моделі підтвердив, що precision, recall та f1-score дорівнюють 1.0 для обох класів. Вибір найкращої моделі ґрунтувався на результатах метрик, і найкращою виявилася Logistic Regression. Прогноз для 10 випадкових зразків з тестової вибірки показав правильні передбачення класів [0, 1, 0, 0, 1, 0, 1, 0, 1, 0].